In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
interval = 2

url = "https://sgis.kostat.go.kr/view/catchmentArea/main#"

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")

browser = webdriver.Chrome(options=options)
browser.get(url)

# res = requests.get(url)
# res.raise_for_status()

# 윈도우창 최대화
browser.maximize_window()

In [2]:
# 팝업창 "취소" 클릭
cancel = browser.find_element_by_class_name("ques02")
cancel.click()

In [3]:
soup = BeautifulSoup(browser.page_source, "lxml")

In [4]:
# 행정구역 "도" 클릭
province = browser.find_element_by_class_name("selct_91")
province.click()
time.sleep(interval)

In [5]:
# 서울특별시 선택
seoul = browser.find_element_by_css_selector("#sido > option:nth-child(1)")
seoul.click()
time.sleep(interval)

In [6]:
# 행정구역 "구" 클릭
districts = browser.find_element_by_class_name("selct_92")
districts.click()
time.sleep(interval)

In [7]:
# 웹 크롤링 및 엑셀파일 만들기
import pandas as pd

# 데이터가 필요한 구 추가 및 제거
districts_list = ["도봉구", "노원구", "강북구", "성북구", "중랑구", "동대문구", "성동구", "광진구"]
districts_list.sort()

# 데이터가 필요한 편의시설 추가 및 제거
categories = ["지하철", "은행", "대형마트", "전통시장", "병원", "응급의료기관", "공원", "스포츠센터", "보건소", "사회복지시설", "행정기관", "백화점"] # 필요한 시설 추가하기

# 빈 딕셔너리 생성
result = {}

for district in districts_list:
    select_district = browser.find_element_by_xpath(f"//option[contains(text(),'{district}')]")
    select_district.click()
    time.sleep(1)
    
    # 각 구별 빈 리스트 생성
    result[district] = []
    
    for category in categories:
        
        # 편의시설 분류 창 스크롤을 내리기
        small_window = browser.find_element_by_id("mCSB_1")
        browser.execute_script("arguments[0].scrollBy(0,400)", small_window)

        # 리스트 내 카테고리 선택 및 클릭
        select_category = browser.find_element_by_xpath(f"//a[contains(text(),'{category}')]")
        select_category.click()
        time.sleep(interval)
        soup = BeautifulSoup(browser.page_source, "lxml")
        category_data = soup.select_one("ul.res_li") 
        
        category_num = 0
        
        for item in category_data :

            # 편의시설 별 데이터 추출
            category_name = item.select('li > span[class="txt02"]')
            
            if category_name != []:
                category_name = item.select('li > span[class="txt02"]')[0].get_text()
                
                # 편의시설 개수 카운팅
                category_num += 1
            else :
                category_name = None
        
        # 편의시설 별 개수를 리스트에 추가
        result[district].append(category_num)
        print(f'{district} {category_name}: {category_num}개')         
#        time.sleep(interval) - 컴퓨터 사양에 따라 시간 늘리기, 기본은 2초

        # 카테고리로 돌아가기 위해 뒤로가기 버튼 클릭
        backToCategory = browser.find_element_by_id("mapLocation_2")
        backToCategory.click()
        time.sleep(interval)
#     print(f'{district}를 완료했습니다.')
    print("="*30)
print("웹 크롤링을 종료합니다.")

강북구 지하철: 4개
강북구 은행: 29개
강북구 대형마트: 1개
강북구 전통시장: 11개
강북구 병원: 17개
강북구 None: 0개
강북구 공원: 3개
강북구 스포츠센터: 193개
강북구 보건소: 3개
강북구 사회복지시설: 66개
강북구 행정기관: 17개
강북구 백화점: 1개
광진구 지하철: 10개
광진구 은행: 56개
광진구 대형마트: 2개
광진구 전통시장: 10개
광진구 병원: 12개
광진구 응급의료기관: 4개
광진구 공원: 3개
광진구 스포츠센터: 265개
광진구 보건소: 3개
광진구 사회복지시설: 62개
광진구 행정기관: 16개
광진구 백화점: 1개
노원구 지하철: 14개
노원구 은행: 58개
노원구 대형마트: 4개
노원구 전통시장: 2개
노원구 병원: 18개
노원구 응급의료기관: 6개
노원구 None: 0개
노원구 스포츠센터: 341개
노원구 보건소: 3개
노원구 사회복지시설: 106개
노원구 행정기관: 20개
노원구 백화점: 1개
도봉구 지하철: 6개
도봉구 은행: 25개
도봉구 대형마트: 3개
도봉구 전통시장: 5개
도봉구 병원: 17개
도봉구 응급의료기관: 2개
도봉구 공원: 2개
도봉구 스포츠센터: 212개
도봉구 보건소: 2개
도봉구 사회복지시설: 65개
도봉구 행정기관: 15개
도봉구 None: 0개
동대문구 지하철: 8개
동대문구 은행: 52개
동대문구 대형마트: 2개
동대문구 전통시장: 19개
동대문구 병원: 26개
동대문구 응급의료기관: 8개
동대문구 공원: 1개
동대문구 스포츠센터: 238개
동대문구 보건소: 2개
동대문구 사회복지시설: 75개
동대문구 행정기관: 15개
동대문구 백화점: 1개
성동구 지하철: 17개
성동구 은행: 55개
성동구 대형마트: 3개
성동구 전통시장: 4개
성동구 병원: 11개
성동구 응급의료기관: 2개
성동구 공원: 4개
성동구 스포츠센터: 214개
성동구 보건소: 3개
성동구 사회복지시설: 66개
성동구 행정기관: 18개
성동구 None: 0개
성북구 지하철: 9개
성북구 은행: 57개
성북구 대형마

In [8]:
# 웹드라이버 종료
print("웹드라이버를 종료합니다.")
quit()

웹드라이버를 종료합니다.


In [9]:
# 카테고리를 인덱스로 갖는 데이터프레임 생성
df = pd.DataFrame(result, index=categories)
df

,강북구,광진구,노원구,도봉구,동대문구,성동구,성북구,중랑구
지하철,4,10,14,6,8,17,9,10
은행,29,56,58,25,52,55,57,31
대형마트,1,2,4,3,2,3,3,6
전통시장,11,10,2,5,19,4,9,11
병원,17,12,18,17,26,11,14,16
응급의료기관,0,4,6,2,8,2,2,6
공원,3,3,0,2,1,4,2,5
스포츠센터,193,265,341,212,238,214,237,293
보건소,3,3,3,2,2,3,2,2
사회복지시설,66,62,106,65,75,66,115,99


In [10]:
# 행과 열을 전환
df.transpose()

,지하철,은행,대형마트,전통시장,병원,응급의료기관,공원,스포츠센터,보건소,사회복지시설,행정기관,백화점
강북구,4,29,1,11,17,0,3,193,3,66,17,1
광진구,10,56,2,10,12,4,3,265,3,62,16,1
노원구,14,58,4,2,18,6,0,341,3,106,20,1
도봉구,6,25,3,5,17,2,2,212,2,65,15,0
동대문구,8,52,2,19,26,8,1,238,2,75,15,1
성동구,17,55,3,4,11,2,4,214,3,66,18,0
성북구,9,57,3,9,14,2,2,237,2,115,21,1
중랑구,10,31,6,11,16,6,5,293,2,99,17,0
